## Make a request to a ChatGPT model

The following example is from:  https://platform.openai.com/docs/quickstart?context=python


In [1]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="In the land of code, there lies a wondrous art,\nA looping dance that sets the soul apart.\nRecursion's melody, a tune of endless tone,\nA function that calls itself, in a harmonious zone.\n\nLike a mirror reflecting an image clear,\nRecursion looks back, wiping out all fear.\nA task divided, a problem sliced thin,\nEach step repeated, a new cycle begins.\n\nThrough the depths of stacks, it wanders deep,\nA journey through layers, secrets to keep.\nInfinite beauty, in its elegant grace,\nA cycle of patterns, in a recursive embrace.\n\nA loop within a loop, a dream unfurled,\nA mystical dance in the coding world.\nSo let recursion guide you, with its magical art,\nAnd unlock the wonders of the programmer's heart.", role='assistant', function_call=None, tool_calls=None)


## Try generating related tweets

In [26]:
import pandas as pd
df_train = pd.read_csv("./data/train_clean_v03.csv")
train_ids = df_train['id']
print(f"shape of base data with spillovers fixed and duplicates removed: {df_train.shape}")
df_train_class0 = df_train.loc[df_train['target'] == 0, ['id', 'text', 'target']]
df_train_class1 = df_train.loc[df_train['target'] == 1, ['id', 'text', 'target']]
print(f"shape of same base data with target == 0: {df_train_class0.shape}")
print(f"shape of same base data with target == 1: {df_train_class1.shape}")
print()

data = {'id': [], 'text': [], 'target': []}
df_aug_class0 = pd.DataFrame(data, columns = ['id', 'text', 'target'])
df_aug_class1 = pd.DataFrame(data, columns = ['id', 'text', 'target'])

for i, row in df_train_class1.iterrows():
    if i < 5:
        print(f"index={i} | {row['id']},{row['text']},{row['target']}")
    df_aug_class1.loc[len(df_aug_class1.index)] = [row['id'], row['text'], row['target']]

print(f"shape of df_aug_class1: {df_aug_class1.shape}")
print()
df_aug_class1.head(10)

shape of base data with spillovers fixed and duplicates removed: (7485, 5)
shape of same base data with target == 0: (4297, 3)
shape of same base data with target == 1: (3188, 3)

index=0 | 1,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
index=1 | 4,Forest fire near La Ronge Sask. Canada,1
index=2 | 5,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
index=3 | 6,13,000 people receive #wildfires evacuation orders in California ,1
index=4 | 7,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ,1
shape of df_aug_class1: (3188, 3)



,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
5,8,#RockyFire Update => California Hwy. 20 closed...,1
6,10,#flood #disaster Heavy rain causes flash flood...,1
7,13,I'm on top of the hill and I can see a fire in...,1
8,14,There's an emergency evacuation happening now ...,1
9,15,I'm afraid that the tornado is coming to our a...,1


In [27]:
base_tweet = df_train_class1['text'].loc[df_train_class1['id'] == 56].values[0]
print(type(base_tweet), base_tweet)

<class 'str'> Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J


In [30]:
# set up a single request
context = "You are a fiction writer who has observed a disaster and likes to tweet."
start_prompt_class1 = "Write me a tweet similar to this one, under 141 characters, " + \
                      "does not use contractions, but refers to a different disaster and location: "
complete_prompt = start_prompt_class1 + base_tweet
print(complete_prompt)

Write me a tweet similar to this one, under 141 characters, does not use contractions, but refers to a different disaster and location: Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J


In [36]:
df_aug_class1_test3 = df_aug_class1.iloc[:3]
print(context)
df_aug_class1_test3

You are a fiction writer who has observed a disaster and likes to tweet.


,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1


In [39]:
from openai import OpenAI
client = OpenAI()

aug_responses = {}

for i, row in df_aug_class1_test3.iterrows():
    prompt_content = start_prompt_class1 + row['text']
    print(f"*** {prompt_content} ***")
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": context},
        {"role": "user", "content": prompt_content}
      ]
    )
    
    aug_responses['id'] = completion
    print(">>>>>>>>>>>>>>>>>>>> chatgpt response <<<<<<<<<<<<<<<<<<<<<")
    print(completion.choices[0].message)


*** Write me a tweet similar to this one, under 141 characters, does not use contractions, but refers to a different disaster and location: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all ***
>>>>>>>>>>>>>>>>>>>> chatgpt response <<<<<<<<<<<<<<<<<<<<<
ChatCompletionMessage(content="The fire's fierce embrace scorched the town's heart. May providence grant solace to those touched by this tragedy. #wildfire #hope", role='assistant', function_call=None, tool_calls=None)
*** Write me a tweet similar to this one, under 141 characters, does not use contractions, but refers to a different disaster and location: Forest fire near La Ronge Sask. Canada ***
>>>>>>>>>>>>>>>>>>>> chatgpt response <<<<<<<<<<<<<<<<<<<<<
ChatCompletionMessage(content='"Devastating forest fire raging near La Ronge, Saskatchewan, Canada. Our thoughts are with the brave firefighters and affected communities. #PrayforLaRonge #ForestFire"', role='assistant', function_call=None, tool_calls=None)
*** Wri

In [41]:
import pickle

pickle_file = './data/first3_class1.pickle'
# Store data (serialize)
with open(pickle_file, 'wb') as handle:
    pickle.dump(aug_responses, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize)
with open(pickle_file, 'rb') as handle:
    unserialized_data = pickle.load(handle)

print(aug_responses == unserialized_data)

True
